### Madina Hayva Putri - 5220411187
### Pemrosesan Teks - B

# Scraping Komentar Youtube Berdasarkan Kata Kunci

In [19]:
!pip install google-api-python-client
!pip install tqdm

In [20]:
from google.colab import userdata
from googleapiclient.discovery import build
import pandas as pd
from tqdm import tqdm

In [29]:
api_key = userdata.get('YouTube')  # ganti nama secret dengan nama yang sama di tab "Rahasia"
youtube = build('youtube', 'v3', developerKey=api_key)

print("Berhasil")

Berhasil


In [40]:
# parameter pencarian video dengan kata kunci
query = "review makanan tiktokers"
max_komen_per_video = 150
min_total_komen = 1000

In [41]:
videos = []
next_page_token = None

print("Mengambil daftar video berdasarkan kata kunci...")
while len(videos) < (min_total_komen // max_komen_per_video) + 5:
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        maxResults=30,
        pageToken=next_page_token
    )
    response = request.execute()
    videos += response["items"]
    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break

print(f"Ditemukan {len(videos)} video terkait '{query}'\n")


Mengambil daftar video berdasarkan kata kunci...
Ditemukan 30 video terkait 'review makanan tiktokers'



In [45]:
data_komen = []
total_komen = 0

for video in tqdm(videos, desc="Mengambil komentar"):
    video_id = video["id"]["videoId"]
    channel_name = video["snippet"]["channelTitle"]
    video_title = video["snippet"]["title"]

    next_page_token = None
    count = 0

    while count < max_komen_per_video:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]
            data_komen.append({
                "nama_channel": channel_name,
                "judul_video": video_title,
                "komentar": comment["textDisplay"],
                "jumlah_like_komentar": comment["likeCount"],
                "tanggal_posting": comment["publishedAt"]
            })
            count += 1
            total_komen += 1
            if count >= max_komen_per_video:
                break  # hanya berhenti per video

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break  # komentar video habis

print(f"\nTotal komentar terkumpul: {len(data_komen)}")

Mengambil komentar: 100%|██████████| 30/30 [00:05<00:00,  5.20it/s]


Total komentar terkumpul: 4069


## Casefolding

In [47]:
df = pd.DataFrame(data_komen)
df["komentar"] = df["komentar"].str.lower() #mengubah huruf menjadi kecil
df["komentar"] = df["komentar"].str.strip() #menghapus spasi berlebih

import re
df["komentar"] = df["komentar"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x)) #menghapus karakter non-alfabet

print(f"Total komentar setelah casefolding: {len(df)}")

Total komentar setelah casefolding: 4069


In [50]:
df.tail()

,nama_channel,judul_video,komentar,jumlah_like_komentar,tanggal_posting
4064,Mia Oktavia Chandra,COBAIN REKOMENDASI HACK MAKANAN DARI COMMENT T...,nasi chitato penyelamat waktu ngekost dulu,4,2025-02-12T03:14:17Z
4065,Mia Oktavia Chandra,COBAIN REKOMENDASI HACK MAKANAN DARI COMMENT T...,penasaran roti o ice cream mixue nih cobain j...,935,2025-02-10T18:04:00Z
4066,Mia Oktavia Chandra,COBAIN REKOMENDASI HACK MAKANAN DARI COMMENT T...,ke 4 nih,3,2025-02-10T17:14:36Z
4067,Mia Oktavia Chandra,COBAIN REKOMENDASI HACK MAKANAN DARI COMMENT T...,ke 3 nieh ka dapet apaaa,2,2025-02-10T13:30:45Z
4068,Mia Oktavia Chandra,COBAIN REKOMENDASI HACK MAKANAN DARI COMMENT T...,pingin cobain yg chitato tp lagi diettt huff c...,45,2025-02-10T12:57:01Z


In [52]:
df.to_csv("data_komen_youtube.csv", index=False, encoding="utf-8-sig")

print("File tersimpan sebagai 'youtube_comments.csv'")

File tersimpan sebagai 'youtube_comments.csv'
